In [117]:
import numpy as np
import math

P_max = 0.00015
TX1_Power = 0.00010 # transfer power of TX1 in watt
TX1_fading_samples = 1000 # Number of fading samples in the channel between TX1 and RX1
RX1_Noise_Power = pow(10,-5) # Noise power in watt
TX1_Bandwidth = pow(10,3) # Bandwidth of the communication channel between TX1 and RX1
TX1_communication_time = 0.0003 # Communication time TX1-RX1

Harvesting_power_TX1 = P_max - TX1_Power  # Power that TX1 allocates for energy harvesting
EH_eta = 0.1 # power conversion efficiency
energy_harvesting_time = 0.0004
EH_fading_samples = 1000

TX2_fading_samples = 1000 # Number of fading samples in the channel between TX1 and RX1
RX2_Noise_Power = pow(10,-5) # Noise power in watt
TX2_Bandwidth = pow(10,3) # Bandwidth of the communication channel between TX1 and RX1
TX2_communication_time = 0.0003 # Communication time TX1-RX1



In [118]:
def FadingChannel(TX_Power, fading_samples):# Rayleigh fading model
    # Generate Rayleigh fading samples (complex Gaussian random variables)
    fading = np.random.normal(0, 1, fading_samples) + 1j * np.random.normal(0, 1, fading_samples)
    
    # Calculate received power
    received_power = TX_Power * np.abs(fading)**2
    
    # Calculate average received power
    RX_Power = np.mean(received_power)
    return RX_Power



In [119]:
def calculate_transferred_data(Received_Power, Noise_Power, Bandwidth, communication_time):
    SNR = Received_Power/Noise_Power
    Rate = Bandwidth * math.log2(1+SNR)
    Data = Rate * communication_time
    return SNR, Rate, Data  

In [120]:
def calculate_harvested_power(power, eta, EH_fading_samples):
    Harvested_power = eta * FadingChannel(power, EH_fading_samples)
    return Harvested_power

In [121]:
RX1_Power = FadingChannel(TX1_Power, TX1_fading_samples) # Received power in RX1

[SNR_RX1, Rate_RX1, Data_TX1] = calculate_transferred_data(RX1_Power, RX1_Noise_Power, TX1_Bandwidth, TX1_communication_time)

Harvested_power_TX1 = calculate_harvested_power(Harvesting_power_TX1, EH_eta, EH_fading_samples) # harvested energy at TX2

RX2_Power = FadingChannel(Harvested_power_TX1, TX2_fading_samples) # Received power in RX1

[SNR_RX2, Rate_RX2, Data_TX2] = calculate_transferred_data(RX2_Power, RX2_Noise_Power, TX2_Bandwidth, TX2_communication_time)